In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [14]:
#file_path = 'Data1.xlsx'
file_path = 'Data2.xlsx'
# Crea un diccionario vacío para almacenar los DataFrames
dataframes = {}

# Lee cada hoja del archivo de Excel y la convierte en un DataFrame
for sheet_name in pd.ExcelFile(file_path).sheet_names[:10]:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    dataframes[sheet_name] = df


In [15]:

print(dataframes[pd.ExcelFile(file_path).sheet_names[9]])


          Fecha   x1   x2              Class  Ancho     H/Pix       H.N  \
0    2024-07-02   30   34     period resting      4  0.039344  0.157377   
1    2024-07-02   34   38      period eating      4  0.039344  0.157377   
2    2024-07-02   38   39     period resting      1  0.039344  0.039344   
3    2024-07-02   39   40      period eating      1  0.039344  0.039344   
4    2024-07-02   40   42     period resting      2  0.039344  0.078689   
...         ...  ...  ...                ...    ...       ...       ...   
6105 2024-11-04  591  615  period rumination     24  0.039344  0.944262   
6106 2024-11-04  615  626     period resting     11  0.039344  0.432787   
6107 2024-11-04  626  633  period rumination      7  0.039344  0.275410   
6108 2024-11-04  633  634     period resting      1  0.039344  0.039344   
6109 2024-11-04  634  640      period eating      6  0.039344  0.236066   

                    H                         S.H  
0     00:09:26.557000             00:09:26.5570

In [16]:
# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    # Borra las filas con ancho 0
    df.drop(df.query('Ancho == 0').index, inplace=True)
    # Actualizamos el índice después de eliminar las filas
    df.reset_index(drop=True, inplace=True)
    #elimina columnas
    df.drop(columns=['x1', 'x2', 'H.N', 'H/Pix','H','S.H'], inplace=True)
    

In [17]:
# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    df['H/Pix']=24/610
    df['H.N']= df['Ancho']*df['H/Pix']
    

In [18]:
# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    # Convertir cada fila de horas a timedelta
    df['H'] = pd.to_timedelta(df['H.N'], unit='hours')
    
    

In [19]:
df

,Fecha,Class,Ancho,H/Pix,H.N,H
0,2024-07-02,period resting,4,0.039344,0.157377,0 days 00:09:26.557377048
1,2024-07-02,period eating,4,0.039344,0.157377,0 days 00:09:26.557377048
2,2024-07-02,period resting,1,0.039344,0.039344,0 days 00:02:21.639344262
3,2024-07-02,period eating,1,0.039344,0.039344,0 days 00:02:21.639344262
4,2024-07-02,period resting,2,0.039344,0.078689,0 days 00:04:43.278688524
...,...,...,...,...,...,...
5911,2024-11-04,period rumination,24,0.039344,0.944262,0 days 00:56:39.344262295
5912,2024-11-04,period resting,11,0.039344,0.432787,0 days 00:25:58.032786885
5913,2024-11-04,period rumination,7,0.039344,0.275410,0 days 00:16:31.475409837
5914,2024-11-04,period resting,1,0.039344,0.039344,0 days 00:02:21.639344262


In [20]:
# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    # Agregar las columnas "desde" y "hasta"
    df["Desde"] = ''
    df["Hasta"] = ''

    # Calcular las columnas "desde" y "hasta"
    for i in range(len(df)):
        if i == 0:
            df.at[i, "Desde"] = pd.to_timedelta('00:00:00')
        else:
            start = df.at[i-1, "Hasta"]
            df.at[i, "Desde"] = start

        end = df.at[i, "H"] + df.at[i, "Desde"]
        df.at[i, "Hasta"] = end

   

In [21]:
# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    #elimina columnas
    df.drop(columns=['Ancho','H/Pix', 'H.N'], inplace=True)
    

In [22]:
from datetime import datetime, timedelta

# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    
    hourly_periods = []
    # Iterar sobre cada fila del dataframe original
    for index, row in df.iterrows():
        fecha = row["Fecha"]
        periodo = row["Class"]
        duracion_timedelta = row["H"]
        desde_timedelta = row["Desde"]
        hasta_timedelta = row["Hasta"]

        start_hour = int(desde_timedelta.total_seconds() // 3600)
        end_hour = int(hasta_timedelta.total_seconds() // 3600)

        # Calcular el total de segundos de duración
        total_seconds = duracion_timedelta.total_seconds()

        for hour in range(start_hour, end_hour + 1):
            # Calcular el tiempo en segundos dentro de la hora actual
            current_hour_start = timedelta(hours=hour )
            current_hour_end = timedelta(hours=(hour + 1) )

            # Calcular la intersección de tiempo dentro de la hora actual
            period_start = max(desde_timedelta, current_hour_start)
            period_end = min(hasta_timedelta, current_hour_end)
            period_seconds = (period_end - period_start).total_seconds()

            # Añadir al DataFrame el periodo correspondiente a la hora actual
            hourly_periods.append({
                "Fecha": fecha,
                "Hora": hour % 24,
                "Periodo": periodo,
                "Tiempo": timedelta(seconds=period_seconds),
                "Desde": period_start,
                "Hasta": period_end ,
            })
            
    # Crear un DataFrame a partir de la lista de diccionarios
    hourly_periods_df = pd.DataFrame( hourly_periods)
    hourly_periods_df.drop(columns=['Desde','Hasta'], inplace=True)
    
    # hourly_periods_df['eriodo_count'] = 1
    
    df_grouped = hourly_periods_df.groupby(['Fecha', 'Hora', 'Periodo']).agg({'Tiempo': 'sum'}).reset_index()

    # Agregar la columna de conteo después del groupby
    conteo = hourly_periods_df.groupby(['Fecha', 'Hora', 'Periodo']).size().reset_index(name='Conteo')

    # Combinar los resultados
    df_grouped = pd.merge(df_grouped, conteo[['Fecha', 'Hora', 'Periodo', 'Conteo']], on=['Fecha', 'Hora', 'Periodo'])

    # Crear pivot table para ambas métricas
    df_pivot_tiempo = df_grouped.pivot_table( index=['Fecha', 'Hora'], columns='Periodo',values='Tiempo', fill_value=pd.Timedelta(0)).reset_index()

    df_pivot_conteo = df_grouped.pivot_table( index=['Fecha', 'Hora'], columns='Periodo', values='Conteo', fill_value=0 ).reset_index()

    # Renombrar las columnas de conteo para diferenciarlas
    conteo_columns = {col: f'{col}_count' for col in df_pivot_conteo.columns if col not in ['Fecha', 'Hora']}
    df_pivot_conteo = df_pivot_conteo.rename(columns=conteo_columns)

    # Combinar ambos DataFrames
    df_final = pd.merge(df_pivot_tiempo, df_pivot_conteo, on=['Fecha', 'Hora'])
    df_final.columns.name = None

    # Define the columns to convert
    time_columns = ['period eating', 'period other', 'period resting', 'period rumination']

    # Convert each timedelta column to decimal hours
    for column in time_columns:
        df_final[column] = (df_final[column].dt.total_seconds() / 3600).round(3)

    # Mostrar el DataFrame resultante
    dataframes[sheet_name] = df_final

In [23]:
import re
# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    namecsv = re.findall(r'\d+', sheet_name)[0]
    
    #df.to_csv(f'Data1_procesados/{namecsv}.csv', index=False)
    df.to_csv(f'Data2_procesados/{namecsv}.csv', index=False)